# www http://brainbashers.com/skyscrapers.asp

## get e.g. a  puzzle from ^^^
### and check there your solution: ==> green square if ok

#example end of this file

# for easy use,
one has to define an input manor for the scy-scraper puzzle

# my choice: 
the numbers from Upside, Right, Down and Left have to be given  as wel the size (size x size)

# use the helper function: <font color ='red'> maakStartVals </font>

# if numbers inside are given 
you have to give them to the program too

# use my <font color = 'red'> SudokuBaseClass </font>
It delivers the z3-conditions for puzzles of size size, see. 
Taking care of importing z3.

### see In[1] 

In [1]:
from SudokuBaseClass import * #see result of this cell ;-)

# self. because the result of maakStartVals is later used in th solver class
def maakStartVals(size = 4):
    """
        make empty startvalue example for ScyscrInfo
    """
    w = ['self.U', 'self.R', 'self.D', 'self.L']
    result = []
    for LL in w:
        tmp = LL + ';'
        for i in range(size - 1):
            tmp += '0 '
        tmp += '0'
        result.append(tmp)
    result.append('self.instance;x,y,w 0,0,0') 
    print('zero based counting! first column is col number 0 etc.')
    print('x,y zero based coordinates w given border value')                 
    print('if 2 or more bordervalues are given use ONE space as seperator token')
    print('have a look of the possible examples, in this notebook')
    return result

SC = SudokuConditions()


from z3 import * #done!
A, cells_c, rows_c, cols_c , instance NOW in SC available


# maybe start here adjusting size and the if ... ;-)

In [2]:
help(maakStartVals)

Help on function maakStartVals in module __main__:

maakStartVals(size=4)
    make empty startvalue example for ScyscrInfo



In [3]:
maakStartVals()

zero based counting! first column is col number 0 etc.
x,y zero based coordinates w given border value
if 2 or more bordervalues are given use ONE space as seperator token
have a look of the possible examples, in this notebook


['self.U;0 0 0 0',
 'self.R;0 0 0 0',
 'self.D;0 0 0 0',
 'self.L;0 0 0 0',
 'self.instance;x,y,w 0,0,0']

In [4]:
size = 4 #for one of three examples in this notebook

# use Helper maakStartVals for describing the given borderconditions and inside numbers


ScyscrInfo = []
if size == 8:
    ScyscrInfo = ['self.U;3 3 0 0 2 3 2 4',
            'self.R;0 0 0 3 2 0 0 3',
            'self.D;5 0 0 0 0 0 3 2',
            'self.L;3 3 0 0 0 0 2 0',
            'self.instance;0,7,4 1,6,1 2,0,6 3,4,1 3,5,7 4,2,2 5,3,6 6,6,5'
            ]

elif size == 3:
    ScyscrInfo = ['self.U;3 2 1',
          'self.R;0 2 2',
          'self.D;0 0 0',
          'self.L;0 0 0',
          'self.instance;1,0,2'
         ]
elif size == 4:
    ScyscrInfo = ['self.U;3 0 3 0', 'self.R;0 1 0 0', 'self.D;0 0 0 0', 'self.L;0 0 3 0', 'self.instance:']
size, ScyscrInfo

(4,
 ['self.U;3 0 3 0',
  'self.R;0 1 0 0',
  'self.D;0 0 0 0',
  'self.L;0 0 3 0',
  'self.instance:'])

In [5]:
class z3_PKHG_Scyscraper_Solver:
    """
    two parameters have to be prepared to use this class
    the size of the puzzle (size = ...) has to be set
    the Scyscraper-infos see the maakStartVals function to build ScyscrInfo 
    """
    def  __init__(self, begin_data = [], size=size, ScyscrInfo=ScyscrInfo):
        self.SC = SudokuConditions(size)
        self.A =  self.SC.A
        self.ScyscrInfo = ScyscrInfo
        self.size = size
        self.cells_c = self.SC.cells_c
        self.rows_c = self.SC.rows_c
        self.cols_c = self.SC.cols_c
        self.instance = self.SC.instance
        self.begin_data = begin_data
        self.allemijneresults = []
        self.U, self.R, self.D, self.L = self.makeRanden()
        self.res_use_all_all = self.use_all_all()
    
        self.endit = self.givenElements()
        if self.begin_data == []:
            self.begin_data = self.makeConditionsNEW()
            for i, data in enumerate(self.begin_data):
                tmpvwk = self.verwerk_defCLASS_c(data)
                self.allemijneresults.append(tmpvwk) #self.verwerk_defCLASS_c(data))
   
        self.instance_c = [ If(self.instance[i][j] == 0,
                  True,
                  self.A[i][j] == self.instance[i][j])
               for i in range(size) for j in range(size) ]
        self.stDD = self.cells_c + self.rows_c + self.cols_c + self.instance_c
        self.scyscraper_c = self.allemijneresults
        #res = self.verwerk_defCLASS_c(self.begin_data)
        
        
        
    def myMax(self, A, B):
         return If(A > B, A, B)
       
    def verwerk_defCLASS_c(self, bgi):
        l_As, l_val = bgi
        result = ['1', 'If(' +  l_As[1] + ' > ' + l_As[0] + ', 1, 0)']
        
        tmp = result[1]
        newResult = [1, eval(tmp)]
        helper = 'self.myMax(' + l_As[0] + ',' + l_As[1] + ')'
        
        
        for i, el in enumerate(l_As[2:]):
            newResult.append(eval('If(' + el + ' > ' + helper + ', 1, 0 )'))
            helper = 'self.myMax(' + l_As[i + 2] + ', ' + helper + ')'
        return Sum(newResult) == l_val
    
    def use_all_all(self, inpu=ScyscrInfo): # ScyscraperInfo was gegeens
        global instance
        result = []
        linpu = len(inpu)
        if inpu[-1] == 'self.instance;':
            #inpu[-1] = 'instance;0,0,0'
            linpu += -1
        for el in inpu[0:linpu] :
            if el == 'self.instance;':
                continue
            res = self.use_one_el_CLASS(el)
            result.append(res)
            if res == True:
                continue
            for i, el in enumerate(res):
                if el == []:
                    continue
                exec(el)
        return result

   
    def use_one_el_CLASS(self, geg):
        instance = self.instance 
        if len(geg) < 3: # each row starts e.g. like thid: 'U; or 'instance
            return 0
        result = []
        firstSplit = geg.split(';')
        sort = firstSplit[0]
        tmp = [] #later needed
        if sort in ['self.U','self.R','self.D','self.L']:
            for el in firstSplit[1:]: #there was only ONE ; in each row
                spsplit = el.split(' ')       # values separated by 1 space
                for i, w in enumerate(spsplit):
                    if w == '0':
                        continue
                    tmp =  sort + '[' +str(i) + '] =  ' + w
                    exec(tmp)
                result.append(tmp) 
        elif sort == 'self.instance':
            if firstSplit[1] == '':
                return True
            data = firstSplit[1].split(' ')   #up to ; is already removed
            for el in data: 
                x, y, wat = el.split(',')
                xx = int(x) + 0
                yy = int(y) + 0
                watw = int(wat) + 0
                instance[xx][yy] = watw  #insert given numbers
            return True          
        return result

    def makeRanden(self, size = size):
        result = []
        randen = ['u','r','d','l']
        for letter in randen:
            capL = 'self.' +letter.upper()
            tmp = capL + ' = [ Int("' + letter + '%s" % (i)) for i in range('+ str(size) + ')]'
            exec(tmp)
            result.append(eval(capL))
        return result

    def givenElements(self):
        elm = ['self.U', 'self.L', 'self.R', 'self.D']
        results = []
        for name in elm:
            tmp = [name]
            obj = eval(name)
            for i, el in enumerate(obj):
                if type(el) == type(1):
                    tmp.append([i, el])
            results.append(tmp)
        return results


    def w_elements_str(self, side, nr, s=size): #rembemer, this is for 6x6 puzzle
        result = []
        if side == 'self.U': #from above to below
            for i in range(s):
                #res = ('AA[' + str(i) + '][' + str(nr) + ']')
                result.append('self.A[' + str(i) + '][' + str(nr) + ']') # U measn from 1 to size downwards colum nr
        elif side == 'self.L':  # from left to right
            for i in range(s):
                #result.append(A[nr][i])
                result.append('self.A[' + str(nr) + '][' + str(i) + ']')
        elif side == 'self.R':  #form right to left
            for i in range(s -1, -1, -1):
                #result.append(A[nr][i]) # U measn from 1 to size downwards colum nr
                result.append('self.A[' + str(nr) + '][' + str(i) + ']')
        elif side == 'self.D':  #form right to left
            for i in range(s -1, -1, -1):
                result.append('self.A[' + str(i) + '][' + str(nr) + ']')
        return result


    def makeConditionsNEW(self):
        result = []
        whatToDo = self.endit #givenElements()
        for what in whatToDo:
            if len(what) > 1:
                z3obj = eval(what[0])
                side_name = what[0]
                #continue
                for doth in what[1:]:
                    tmp = self.w_elements_str(side_name,doth[0]) #, z3obj[doth[1]])
                    tmp1 = [tmp, doth[1]]
                    result.append([tmp, doth[1]])
        return result

    def runthesolver(self, size = size, countmax=2):        
        startdata = self.stDD.copy()  + self.scyscraper_c
        
        count = 0
        countmax = 2 #if there are more than on solution
        z3solver = Solver()
        z3solver.add(startdata)
        while z3solver.check() == sat and count < countmax:
            count += 1
            print('count = ', count)
            m = z3solver.model()
            r = [ [ m.evaluate(self.A[i][j]) for j in range(size) ]  for i in range(size) ]
            for rij in r:
                print(rij)
            vals = [self.A[i][j] == m.evaluate(self.A[i][j]) for j in range(size)  for i in range(size)]
            z3solver.add(Not(And(vals)))
        print("\n====thanks to z3 ;-) =========\n")
        if count == 0:
            print('unsat')
        elif count == 1:
            print('solution found')
        else:
            print('count = {} '.format(count), '\n but more solutions possible!')
    



       

In [6]:
help(z3_PKHG_Scyscraper_Solver) #given values used

Help on class z3_PKHG_Scyscraper_Solver in module __main__:

class z3_PKHG_Scyscraper_Solver(builtins.object)
 |  z3_PKHG_Scyscraper_Solver(begin_data=[], size=4, ScyscrInfo=['self.U;3 0 3 0', 'self.R;0 1 0 0', 'self.D;0 0 0 0', 'self.L;0 0 3 0', 'self.instance:'])
 |  
 |  two parameters have to be prepared to use this class
 |  the size of the puzzle (size = ...) has to be set
 |  the Scyscraper-infos see the maakStartVals function to build ScyscrInfo
 |  
 |  Methods defined here:
 |  
 |  __init__(self, begin_data=[], size=4, ScyscrInfo=['self.U;3 0 3 0', 'self.R;0 1 0 0', 'self.D;0 0 0 0', 'self.L;0 0 3 0', 'self.instance:'])
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  givenElements(self)
 |  
 |  makeConditionsNEW(self)
 |  
 |  makeRanden(self, size=4)
 |  
 |  myMax(self, A, B)
 |  
 |  runthesolver(self, size=4, countmax=2)
 |  
 |  use_all_all(self, inpu=['self.U;3 0 3 0', 'self.R;0 1 0 0', 'self.D;0 0 0 0', 'self.L;0 0 3 0', 'self.instance

In [7]:
MC = z3_PKHG_Scyscraper_Solver()
for el in dir(MC):
    if el.startswith('_'):
        continue
    else:
        print(el)

A, cells_c, rows_c, cols_c , instance NOW in SC available
A
D
L
R
SC
ScyscrInfo
U
allemijneresults
begin_data
cells_c
cols_c
endit
givenElements
instance
instance_c
makeConditionsNEW
makeRanden
myMax
res_use_all_all
rows_c
runthesolver
scyscraper_c
size
stDD
use_all_all
use_one_el_CLASS
verwerk_defCLASS_c
w_elements_str


In [8]:
# %timeit MC.runthesolver()
MC.runthesolver()

count =  1
[2, 4, 1, 3]
[3, 1, 2, 4]
[1, 3, 4, 2]
[4, 2, 3, 1]

====thanks to z3 ;-) =========

solution found


# <font color ='red'>timeit used gives:</font>
## 8x8 puzzle
solution found
1.99 s ± 321 m
s per loop (mean ± std. dev. of 7 runs, 1 loop each)


solution found
2.65 s ± 623 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## 4x4 puzzle:

solution found
73.6 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

solution found
65.6 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

# 4x4 test 31 src contains date
# try this one, (my first 6x6 solved) ;-)
<img src="sol6x6_14aug.png" />

<img src='sol4x4_31aug.png'/>

<img src="sol8x8_20aug.png"/>